# Projet maison n° 4

In [1]:
# imports
import pandas as pd
import os
import re
import glob

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas, pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [2]:
# names us
def df_names_us():
    column_names = ['year', 'name', 'gender', 'births']
    files_to_read = glob.glob(os.path.join(os.getcwd(),'names') + '\yob*.txt')
    all_lines = []
    for file in files_to_read:
        base_name = os.path.basename(file)
        match = re.fullmatch('yob(.*)\.txt',base_name)
        year = match.groups()[0]
        with open(file,'r') as f:
            for line in f:
                line_data = line.rstrip().split(',')
                line_data.insert(0,year)
                all_lines.append(line_data)
    df = pd.DataFrame(all_lines, columns = column_names)
    return df

df = df_names_us()
df

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746
...,...,...,...,...
1989396,2019,Zyheem,M,5
1989397,2019,Zykel,M,5
1989398,2019,Zyking,M,5
1989399,2019,Zyn,M,5


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2019_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [60]:
# names fr
def df_names_fr():
    # Ouverture du fichier
    df = pd.read_csv('nat2019.csv',sep = ';')
    
    # Réarrangement selon les titres des colonnes
    df = df[['annais','preusuel','sexe','nombre']]
    df.columns = ['year', 'name', 'gender', 'births']
    
    # Modification des valeurs pour Gender
    mapping_gender = {2: 'F', 1: 'M'}
    df['gender'] = df['gender'].map(mapping_gender)
    
    # suppression de toutes les lignes avec un NaN et des lignes contenant XXXX
    df.dropna()
    df = df.loc[~df['year'].str.contains('XXXX')]
    
    # Nettoyage des prénoms : suppression de _PRENOMS_RARES, des prénoms de moins de 2 lettres
    # Initiales en majuscule, autres lettres en minuscule
    df = df.loc[(df['name'].astype(str) != '_PRENOMS_RARES') & (df['name'].apply(lambda x:len(str(x)) >= 2))]
    
    # Fonction pour réécrire les prénoms dans le format names_US, gestion des prénoms composés
    def rewrite_names(x):
        name = str(x)
        if '-' in name:
            name_parts = name.split('-')
            new_name = name_parts[0][0] + name_parts[0][1:].lower()
            for i, part in enumerate(name_parts[1:]):
                new_name += '-' + name_parts[1:][i][0] + name_parts[1:][i][1:].lower()
        else:
            new_name = name[0] + name[1:].lower()
        return new_name
    
    df['name'] = df['name'].apply(rewrite_names)
    
    #Sort the values obtained
    df = df.sort_values(by = ['year','gender','births','name'], ascending = [True,False,False,True])
    
    df["births"] = df["births"].astype(object)
    df = df.reset_index(drop=True)
    
    return df

#TEST
df = df_names_fr()
df

,year,name,gender,births
0,1900,Jean,M,14097
1,1900,Louis,M,9051
2,1900,Pierre,M,7456
3,1900,Joseph,M,7259
4,1900,Henri,M,6921
...,...,...,...,...
615907,2019,Élizabeth,F,3
615908,2019,Élyssa,F,3
615909,2019,Énorah,F,3
615910,2019,Éya,F,3


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Télécharger le dataset des taux de change : http://webstat.banque-france.fr/fr/downloadFile.do?id=5385698&exportType=csv


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [163]:
# taux de change
def df_taux_change(devises):
    df = pd.read_csv('Webstat_Export_20201031.csv',sep=';')
    df.columns = [df.columns[0]] + [re.fullmatch('^.* \((.*)\)$',col).groups()[0] for col in df.columns[1:]]

    df = df[[df.columns[0]] + [col for col in df.columns if any(col == dev for dev in devises)]].iloc[5:]

    df['Titre :'] = pd.to_datetime(df['Titre :'], format = '%d/%m/%Y')
    df = df.set_index('Titre :' ,drop=True)
    df.dropna()
    df.index.name = None
    df = df.apply(lambda x:x.str.replace(',','.'))
    
    for index, series in df.iterrows():
        if any(series.str.contains('-')):
            df = df.drop(index)
            
    df = df.apply(lambda x:x.astype(float))
    
    return df

#TEST
devises = ['CHF', 'GBP', 'USD']
df = df_taux_change(devises)
df

,CHF,GBP,USD
2020-10-30,1.0698,0.90208,1.1698
2020-10-29,1.0684,0.90430,1.1704
2020-10-28,1.0693,0.90662,1.1727
2020-10-27,1.0732,0.90718,1.1832
2020-10-26,1.0719,0.90755,1.1819
...,...,...,...
1999-01-08,1.6138,0.70940,1.1659
1999-01-07,1.6165,0.70585,1.1632
1999-01-06,1.6116,0.70760,1.1743
1999-01-05,1.6123,0.71220,1.1790


### Tests

In [160]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 1989401)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 615912)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [161]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [162]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 18.119s

OK
